# Outline experiment final assignment

## Presentation of a story

1. Multiple pages of text on screen. Participants can view a page as long as they want, and press the spacebar to proceed to the next page.
2. Text and background will be in grayscale color. It will be selected at random what the contrast between text and background will be.
3. Contrast between text and background per participant will have to be stored.
4. Reading time per page is collected.
5. filenames have to be dependend on participant name

## Questionnaires about reading experiences

1. Story World Absorption Scale (SWAS) will be presented.
2. Participants will have to answer each question on a scale of 1-7. Other responses result in an NA
3. Output from the questionnaires will be collected.

## Analysis
1. Calculate mean per subscale for SWAS
2. Calculate total reading time per participant
3. Calculate correlation between SWAS & total reading time
4. Analyze whether means for SWAS subscales and total reading time is related to level of contrast between text and background.

## Visualization
1. Visualize correlation SWAS & total reading time
2. Visualize Mean per subscale SWAS per level of contrast
3. Visualize total reading time per level of contrast

# Step 1
## Presentation of a story

1. Multiple pages of text on screen. Participants can view a page as long as they want, and press the spacebar to proceed to the next page.
2. Text and background will be in grayscale color. It will be selected at random what the contrast between text and background will be.
3. Contrast between text and background per participant will have to be stored.
4. Reading time per page is collected.
5. filenames have to be dependend on participant name

In [63]:
import subprocess as sp
sp.run(['python', 'storypresentation.py'])

CompletedProcess(args=['python', 'storypresentation.py'], returncode=0)

# Step 2
## Questionnaires about reading experiences
1. Story World Absorption Scale (SWAS) will be presented.
2. Participants will have to answer each question on a scale of 1-7. Other responses result in an NA
3. Output from the questionnaires will be collected.

In [69]:
import subprocess as sp
sp.run(['python', 'questionnaires.py'])

CompletedProcess(args=['python', 'questionnaires.py'], returncode=0)

# Step 3
## both experiment and questionnaires in one main script. 
### In this script I want to:
1. open window for experiment
2. ask participant number
3. run experiment
4. run questionnaires
5. close window

### NB: these steps can than be removed from the sub-scripts!

In [79]:
import subprocess as sp
sp.run(['python', 'main.py'])

CompletedProcess(args=['python', 'main.py'], returncode=0)

In [71]:
import re
ppn = ['s','3','23']
ppn = ''.join(ppn)
ppn = re.findall("\d+",ppn)
print(ppn)

  File "main.py", line 43
    !python storypresentation.py
    ^
SyntaxError: invalid syntax
